# PMTiles med Python

Dette er et eksempel som bruker Python-biblioteket `pmtiles` til å lage PMTiles. En fil for raster, som henter bilder fra WMS og en for vektor, som generer tilfeldige punkter med egenskaper som inneholder farge og radius.

In [ ]:
%pip install mapbox-vector-tile mercantile pmtiles owslib shapely

## Oppsett

In [ ]:
from pathlib import Path

import mercantile
from pmtiles.tile import Compression, TileType
import shapely

base_dir = Path(".", "pythonic")

lng = 10.248942
lat = 60.143889
min_zoom = 9
max_zoom = 11

tiles = [
    mercantile.tile(lng, lat, min_zoom),
]

for i in range(min_zoom, max_zoom):
    zoom = i + 1
    tiles += mercantile.children(tiles[0], zoom=zoom)

polys = shapely.MultiPolygon([shapely.box(*mercantile.bounds(tile)) for tile in tiles])
bounds = polys.bounds
center = polys.centroid

e7 = 10000000.0

header = {
    "tile_type": TileType.UNKNOWN,
    "tile_compression": Compression.NONE,
    "min_zoom": min_zoom,
    "max_zoom": max_zoom,
    "min_lon_e7": int(bounds[0] * e7),
    "min_lat_e7": int(bounds[1] * e7),
    "max_lon_e7": int(bounds[2] * e7),
    "max_lat_e7": int(bounds[3] * e7),
    "center_zoom": min_zoom,
    "center_lat_e7": int(center.x * e7),
    "center_lon_e7": int(center.y * e7),
}

header

{'tile_type': <TileType.UNKNOWN: 0>,
 'tile_compression': <Compression.NONE: 1>,
 'min_zoom': 9,
 'max_zoom': 11,
 'min_lon_e7': 98437500,
 'min_lat_e7': 598889368,
 'max_lon_e7': 105468750,
 'max_lat_e7': 602398111,
 'center_zoom': 9,
 'center_lat_e7': 101953125,
 'center_lon_e7': 600643740}

## Raster

Hent ned tiles fra WMS og skriv disse til PMTiles.

In [39]:
from pmtiles.tile import TileType, zxy_to_tileid
from pmtiles.writer import Writer
from owslib.wms import WebMapService

wms = WebMapService("https://wms.geonorge.no/skwms1/wms.topograatone")
main_layer = list(wms.contents.keys())[0]

with base_dir.joinpath("raster.pmtiles").open("wb") as f:
    writer = Writer(f)
    
    for tile in tiles:
        bounds = mercantile.bounds(tile)
        res = wms.getmap(
            layers=[main_layer],
            srs="EPSG:4326",
            bbox=bounds,
            size=(256, 256),
            format='image/png',
            transparent=True,
        )

        tileid = zxy_to_tileid(tile.z, tile.x, tile.y)
        writer.write_tile(tileid, res.read())
    
    writer.finalize(header | {"tile_type": TileType.PNG}, {"attribution": "Kartverket"})

## Vektor

Generer tilfeldige punkter innenfor en tile og gi disse hver sin farge i *properties*.

In [41]:
import random

import mapbox_vector_tile
from pmtiles.tile import TileType, zxy_to_tileid
from pmtiles.writer import Writer
import shapely

colors = [
    "#9e0142",
    "#d53e4f",
    "#f46d43",
    "#fdae61",
    "#fee08b",
    "#e6f598",
    "#abdda4",
    "#66c2a5",
    "#3288bd",
    "#5e4fa2",
]

with base_dir.joinpath("vector.pmtiles").open("wb") as f:
    writer = Writer(f)

    for tile in tiles:
        features = []
        for color in colors:
            x = random.randint(0, 4096)
            y = random.randint(0, 4096)
            geometry = shapely.Point(x, y)
            features.append({
                "geometry": geometry.wkb,
                "properties": {
                    "color": color,
                    "radius": random.randint(10, 30),
                }
            })
        layer = {
            "name": "points",
            "features": features,
        }
        data = mapbox_vector_tile.encode([layer])
        
        tileid = zxy_to_tileid(tile.z, tile.x, tile.y)
        writer.write_tile(tileid, data)
    
    writer.finalize(header | {"tile_type": TileType.MVT}, {"attribution": "Kartverket"})

## Script

Kjør denne for å generere en JavaScript-fil som registrerer PMTiles-protokollen på MapLibre og laster kartet med posisjon og zoom-begrensninger i henhold til tilene som er generert.

Kjør så en web-server fra *pythonic*-katalogen, på port `5173`.

### Eksempel

```sh
cd pythonic
python -m http.server 5173
```

...eller...

```sh
npm install --global http-server
cd pythonic
npx http-server --port 5173
```

In [44]:
from string import Template

tpl = Template("""
const protocol = new pmtiles.Protocol();
maplibregl.addProtocol('pmtiles', protocol.tile);

const map = new maplibregl.Map({
    container: 'map',
    style: 'http://localhost:5173/styles.json',
    center: [$lng, $lat],
    maxZoom: $max_zoom,
    minZoom: $min_zoom,
    zoom: $min_zoom,
});
""")

txt = tpl.substitute(
    lat=lat,
    lng=lng,
    max_zoom=max_zoom,
    min_zoom=min_zoom,
)

with base_dir.joinpath("main.js").open("w") as f:
    f.write(txt)